In [1]:
# 라이브러리 import
! pip install bs4
! pip install pandas
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import pprint

In [2]:
### 중앙부처 데이터만 저장할 수 있도록 만든 코드입니다. 지자체나 지역별 데이터 저장을 원하면 수정하여 활용하시길 바랍니다.

## 1. 온통 청년 api로 정책 html 불러오기
key = 'f93ad94f49a4829e7739cc9f' ### 여기에 신청한 본인의 key를 넣어야 합니다!
page = ['0', '1', '2', '3', '4'] # 조회할 페이지, 총 정책 수가 업데이트로 인해 늘어나면 현재 '0' ~ '4'인 페이지를 추가해야 할 수도 있음. 예) page = ['0', '1', '2', '3', '4', '5']
outputNum = '100' # 출력건수, 기본 10 최대 100
#region = '003002015' # 지역별로 조회하고 싶다면 지역 코드를 지정하면 됨. 여러 지역을 동시에 조회하고 싶다면 쉼표(,)로 코드를 연결해 주면 됨. 예) '003002015,003002014'

contents = ''
for p in page:
    #queryParams = 'pageIndex=' + page + '&display=' + outputNum + '&openApiVlak=' + key + '&srchPolyBizSecd=' + region # 지역별로 조회하게 되면 page를 더 적게 지정해야 할 수도 있음.
    queryParams = 'pageIndex=' + p + '&display=' + outputNum + '&openApiVlak=' + key
    url = 'https://www.youthcenter.go.kr/opi/youthPlcyList.do?'+ queryParams

    # url 불러오기
    response = requests.get(url)
    #print(type(response))
    print(len(response.text)) # 130으로 나오는 경우 해당 페이지에 출력할 정책이 없는 경우임, 즉 130 이전 page까지만 포함시키면 됨.

    #데이터 값 출력해보기
    contents += response.text
    #print(type(contents)) # string 객체로 바뀜

247000
247000
239051
266696
99221


In [3]:
# 데이터 결과값 예쁘게 출력해주는 코드
pp = pprint.PrettyPrinter(indent=4, depth=None)
print(pp.pprint(contents))

('<?xml version="1.0" encoding="UTF-8"?>\n'
 '<youthPolicyList>\n'
 '   <pageIndex>1</pageIndex>\n'
 '   <totalCnt>337</totalCnt>\n'
 '   <youthPolicy>\n'
 '      <rnum>1</rnum>\n'
 '      <bizId><![CDATA[R2024021319563]]></bizId>\n'
 '      <polyBizSecd>003002009</polyBizSecd>\n'
 '      <polyBizTy>지자체</polyBizTy>\n'
 '      <polyBizSjnm><![CDATA[2024년 강원 소셜 크리에이터 모집 공고]]></polyBizSjnm>\n'
 '      <polyItcnCn><![CDATA[- 소셜 크리에이터 공모 및 선발- 강원특별자치도 소재 콘텐츠 제작 활동 지원 및 도정 '
 '홍보 협업 등]]></polyItcnCn>\n'
 '      <sporCn><![CDATA[- 강원특별자치도 홍보 콘텐츠 제작비 지원 ※도에서 인정한 컨텐츠에 한하여 제작비 지원 (유튜브 '
 '분야) 1년 9편, 편당(롱폼 1 + 숏폼 1) 50만원 (SNS 분야) 월 최대 30만원 ※별도 기준에 따른 차등 지급 - 연말 우수 '
 '활동 크리에이터 시상(강원특별자치도지사상) 4팀]]></sporCn>\n'
 '      <sporScvl><![CDATA[모집규모: 30팀 (유튜브 10팀 / SNS 분야(블로그, 페이스북, 인스타, 카카오스토리, '
 '트위터 등) 20팀]]></sporScvl>\n'
 '      <bizPrdCn><![CDATA[사업기간: 2024.3.~12]]></bizPrdCn>\n'
 '      <prdRpttSecd><![CDATA[002004]]></prdRpttSecd>\n'
 '      <rqutPrdCn><![CDATA[- 모집대상: 강원특별자치도를 주제로 소셜 콘텐츠를 창작할 수 

In [4]:
## 2. BeautifulSoup으로 html parsing
soup = BeautifulSoup(contents, 'html.parser')
#finded_values=soup.find_all('ageInfo')
#[x.text for x in finded_values] # .text를 통해 안의 내용물만을 불러옵니다

/Users/sukyungseok/jingi-chatbot/chatbot-using-langchain/.conda/lib/python3.11/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


In [5]:
print('현재 총 정책 수:', soup.find('totalcnt').text.replace('<totalcnt>', '').replace('</totalcnt>', ''))

현재 총 정책 수: 337


In [6]:
## 3. df 생성
YouthPolicyInfo = {}
# attr 이름 모두 소문자로
#attr_to_find_list=['polybizsjnm','sporcn', 'ageinfo','majrrqiscn','empmsttscn','splzrlmrqiscn','accrrqiscn','prcpcn','aditrscn','prcplmtttrgtcn']
attr_to_find_list=['rnum', 'bizid', 'polybizsecd', 'polybizty', 'polybizsjnm', 'polyitcncn', 'sporcn', 'sporscvl', 'bizprdcn', 'prdrpttsecd', 'rqutprdcn', 
                   'ageinfo','majrrqiscn','empmsttscn','splzrlmrqiscn','accrrqiscn','prcpcn','aditrscn','prcplmtttrgtcn',
                   'rqutproccn', 'pstnpaprcn', 'jdgnprescn', 'rquturla', 'rfcsiteurla1', 'rfcsiteurla2', 'mngtmson', 'mngtmrofchercn', 'cherctpccn',
                   'cnsgnmor', 'tintchercn', 'tintcherctpccn', 'etct', 'polyrlmcd'] # 모두 소문자여야 함.
print(len(attr_to_find_list))

for each_attr in attr_to_find_list:
    finded_attr=soup.find_all(each_attr)
    YouthPolicyInfo[each_attr]=[x.text for x in finded_attr]

df = pd.DataFrame(YouthPolicyInfo)
#alt_colnames = ['정책명', '지원내용', '연령', '전공요건', '취업상태', '특화분화', '학력', '거주지및소득', '추가단서', '참여제한']
alt_colnames = ['번호', '정책 ID', '정책 일련번호', '기관 및 지자체 구분', '정책명', '정책 소개', '지원 내용', '지원 규모', '사업 운영 기간', '사업 신청 기간 반복 구분 코드', '사업 신청 기간', 
                '연령', '전공', '취업 상태', '특화 분화', '학력', '거주지 및 소득', '추가 단서 사항', '참여 제한 대상', 
                '신청 절차', '제출 서류', '심사 발표', '신청 사이트 주소', '참고 사이트 주소1', '참고 사이트 주소2', '주관 부처명', '주관 부처 담당자 이름', '주관 부처 담당자 연락처',
                '운영 기관명', '운영 기관 담당자 이름', '운영 기관 담당자 연락처', '기타 사항', '정책 분야 코드']
print(len(alt_colnames))

df.columns = alt_colnames

33
33


In [7]:
print(df.shape)
df.head()

(437, 33)


,번호,정책 ID,정책 일련번호,기관 및 지자체 구분,정책명,정책 소개,지원 내용,지원 규모,사업 운영 기간,사업 신청 기간 반복 구분 코드,...,참고 사이트 주소1,참고 사이트 주소2,주관 부처명,주관 부처 담당자 이름,주관 부처 담당자 연락처,운영 기관명,운영 기관 담당자 이름,운영 기관 담당자 연락처,기타 사항,정책 분야 코드
0,1,R2024021319563,003002009,지자체,2024년 강원 소셜 크리에이터 모집 공고,- 소셜 크리에이터 공모 및 선발- 강원특별자치도 소재 콘텐츠 제작 활동 지원 및 ...,- 강원특별자치도 홍보 콘텐츠 제작비 지원 ※도에서 인정한 컨텐츠에 한하여 제작비 ...,"모집규모: 30팀 (유튜브 10팀 / SNS 분야(블로그, 페이스북, 인스타, 카카...",사업기간: 2024.3.~12,002004,...,-,-,강원특별자치도 대변인실,강원특별자치도 대변인실,강원특별자치도 대변인실 033-249-2051,강원특별자치도 대변인실,-,-,모집공고문 참조,023050
1,2,R2024020719543,003001001,중앙부처,청년 국가기술자격시험 응시료 지원사업,청년의 구직활동과 경력개발의 경제적 부담을 완화하기 위한 사업,국가기술자격시험 응시료 50% 지원,-,-,002001,...,https://www.q-net.or.kr/man004.do?id=man00402&...,-,고용노동부,-,-,한국산업인력공단,-,-,"1인당 지원한도는 연간 3회이며, 지원예산 소진시 잔여 지원횟수와 상관없이 신청이 ...",023010
2,3,R2024020619508,003002005,지자체,청년 주거급여 지원,취학·구직 등의 사유로 부모와 떨어져 주거지를 달리하는 19세~30세 미만의 미혼청...,"지원상한액(광역시 기준) : 1인 216,000원, 2인 240,000원, 3인 2...",주거급여 수급자 및 주거급여 수급자와 세대분리한 청년,2024.01.01.~2024.12.31.,002004,...,https://www.bokjiro.go.kr/ssis-teu/twoaa/aplyS...,-,광주광역시 도시공간국 주택정책과,김형열 주무관,062-613-4831,"광주광역시, 5개 자치구",-,-,청년명의의 임대차계약을 체결하고 임차료를 지불하는 청년에게 지급(전입신고 필수)|주...,023020
3,4,R2024020619532,003002005,지자체,광주형청년인공지능 일자리매칭프로젝트,광주광역시 소재 ICT/SW/AI 중소/중견 기업의 인력난 해소 및 이에 취업하는 ...,(기업) 신규채용 1인당 최대 월 200만원 인건비 지원 (기업 자부담 10% 이상...,38명,2022.03.01.~2024.12.31.,002004,...,-,-,광주광역시 청년정책관,조천호 주무관,062-613-3581,(재)광주정보문화산업진흥원,-,062-610-2407,-,023010
4,5,R2024020619507,003001038,중앙부처,청년전용창업자금(창업기반 지원),기술과 아이디어가 있지만 자금을 마련할 담보가 부족한 청년 창업기업을 위해 장기·저...,"□ 대출한도\r\n - 연간 1억원 이내\r\n ※ 조업 및 혁신성장 분야, 지역주...",null,null,002001,...,https://www.kosmes.or.kr,null,중소벤처기업진흥공단,null,null,중소벤처기업진흥공단,null,(문의처) 중소벤처기업진흥공단 / 1811-3655,null,023010


In [8]:
## 4. 부족한 정보 추가
# 4-1. 정책 분야 추가
polysec = []
for code in df['정책 분야 코드']:
    if code == '023010':
        polysec.append('일자리 분야')
    elif code == '023020':
        polysec.append('주거 분야')
    elif code == '023030':
        polysec.append('교육 분야')
    elif code == '023040':
        polysec.append('복지·문화 분야')
    elif code == '023050':
        polysec.append('참여·권리 분야')
    else:
        polysec.append('코드 매칭 분야 없음')

df['정책 분야'] = polysec

In [9]:
# 4-2. 온통 청년 url 추가
df['온통 청년 url'] = "https://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifDtl.do?pageIndex=1&frameYn=&bizId=" + df['정책 ID'] + "&dtlOpenYn=&plcyTpOpenTy=&plcyCmprInfo=&srchWord=&chargerOrgCdAll=&srchAge=&trgtJynEmp=&trgtJynEmp=&srchSortOrder=1&pageUnit=12"

In [21]:
print(df.shape)
df.head()

(337, 35)


,번호,정책 ID,정책 일련번호,기관 및 지자체 구분,정책명,정책 소개,지원 내용,지원 규모,사업 운영 기간,사업 신청 기간 반복 구분 코드,...,주관 부처명,주관 부처 담당자 이름,주관 부처 담당자 연락처,운영 기관명,운영 기관 담당자 이름,운영 기관 담당자 연락처,기타 사항,정책 분야 코드,정책 분야,온통 청년 url
0,1,R2024021319563,003002009,지자체,2024년 강원 소셜 크리에이터 모집 공고,- 소셜 크리에이터 공모 및 선발- 강원특별자치도 소재 콘텐츠 제작 활동 지원 및 ...,- 강원특별자치도 홍보 콘텐츠 제작비 지원 ※도에서 인정한 컨텐츠에 한하여 제작비 ...,"모집규모: 30팀 (유튜브 10팀 / SNS 분야(블로그, 페이스북, 인스타, 카카...",사업기간: 2024.3.~12,002004,...,강원특별자치도 대변인실,강원특별자치도 대변인실,강원특별자치도 대변인실 033-249-2051,강원특별자치도 대변인실,-,-,모집공고문 참조,023050,참여·권리 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...
1,2,R2024020719543,003001001,중앙부처,청년 국가기술자격시험 응시료 지원사업,청년의 구직활동과 경력개발의 경제적 부담을 완화하기 위한 사업,국가기술자격시험 응시료 50% 지원,-,-,002001,...,고용노동부,-,-,한국산업인력공단,-,-,"1인당 지원한도는 연간 3회이며, 지원예산 소진시 잔여 지원횟수와 상관없이 신청이 ...",023010,일자리 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...
2,3,R2024020619508,003002005,지자체,청년 주거급여 지원,취학·구직 등의 사유로 부모와 떨어져 주거지를 달리하는 19세~30세 미만의 미혼청...,"지원상한액(광역시 기준) : 1인 216,000원, 2인 240,000원, 3인 2...",주거급여 수급자 및 주거급여 수급자와 세대분리한 청년,2024.01.01.~2024.12.31.,002004,...,광주광역시 도시공간국 주택정책과,김형열 주무관,062-613-4831,"광주광역시, 5개 자치구",-,-,청년명의의 임대차계약을 체결하고 임차료를 지불하는 청년에게 지급(전입신고 필수)|주...,023020,주거 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...
3,4,R2024020619532,003002005,지자체,광주형청년인공지능 일자리매칭프로젝트,광주광역시 소재 ICT/SW/AI 중소/중견 기업의 인력난 해소 및 이에 취업하는 ...,(기업) 신규채용 1인당 최대 월 200만원 인건비 지원 (기업 자부담 10% 이상...,38명,2022.03.01.~2024.12.31.,002004,...,광주광역시 청년정책관,조천호 주무관,062-613-3581,(재)광주정보문화산업진흥원,-,062-610-2407,-,023010,일자리 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...
4,5,R2024020619507,003001038,중앙부처,청년전용창업자금(창업기반 지원),기술과 아이디어가 있지만 자금을 마련할 담보가 부족한 청년 창업기업을 위해 장기·저...,"□ 대출한도\r\n - 연간 1억원 이내\r\n ※ 조업 및 혁신성장 분야, 지역주...",null,null,002001,...,중소벤처기업진흥공단,null,null,중소벤처기업진흥공단,null,(문의처) 중소벤처기업진흥공단 / 1811-3655,null,023010,일자리 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...


In [ ]:
# 정책 분야 카테고리 확인
df['정책 분야'].unique()

In [22]:
## 5. 중복 데이터 제거
# 5-1. 중복 데이터 확인
print(df['정책명'].value_counts())
df['정책명'].unique()

정책명
청년전용 버팀목전세자금                           2
충북행복결혼공제                               2
2024년 강원 소셜 크리에이터 모집 공고                1
자립수당 지급                                1
2024년 청년창업사관학교 14기 모집                  1
                                      ..
2024년 상반기  서울특별시 동부기술교육원 직업교육훈련생 모집    1
2024 재직 청년근로자 근속지원사업                   1
2024년 재외공관 공공외교 현장실습원                  1
2024학년도 천안행복기숙사 1학기 정기 입사생 모집          1
구직청년 면접 정장 대여                          1
Name: count, Length: 335, dtype: int64


array(['2024년 강원 소셜 크리에이터 모집 공고', '청년 국가기술자격시험 응시료 지원사업', '청년 주거급여 지원',
       '광주형청년인공지능 일자리매칭프로젝트', '청년전용창업자금(창업기반 지원)', '청년창업기업 세금 면제',
       '온라인 AI모의면접 확대·운영', '청년농업인 영농정착 지원사업', '제주 청년 취업지원 희망프로젝트 사업',
       '충북행복결혼공제', '재직 청년근로자 근속 지원', '2024년 취업준비생 경쟁력 강화 지원사업(AI취업준비)',
       '취업준비생 레벨업 지원', '2024년 상반기 청년창업팀 모집', '대학생 학자금대출 이자지원 사업',
       '청년내일희망카드', '청년창업카드 지원', '[청년정책담당관]2024년 금상반기 학자 대출이자 지원 사업',
       '제1기 합천군 청년정책네트워크 추가모집', '경기도 청소년 자립두배 통장 참가자 모집',
       '2024년 창업기업 스케일업 지원사업 R&D 신규지원 시행계획 공고',
       '2024년도 재도전성공패키지 (예비)재창업자 모집 공고',
       "'24년 청년식품 창업 패키지(초기창업) 참여기업 모집 공고",
       '2024년 창업성공패키지 청년창업사관학교 14기 입교생 모집 공고', '근로자 휴가비 지원사업',
       '경기 디지털 전환 기술 글로벌화 지원 참가기업 모집', '농식품 창업 아이디어 성장전략 과정 교육생 모집',
       '2024년 농식품 벤처육성 지원사업(창업기업) 모집 공고', '2024년 청년CEO육성사업 청년창업가 모집 공고',
       '2024 성남시 청년 재창업 지원', '2024 충남관광 스타트업 육성지원',
       '2024년 생애최초 청년창업 지원사업 예비창업자 모집 공고(수정)',
       '2024년 스타트업 AI기술인력 양성사업(이어드림 스쿨) 청년인재 모집공고',
       '(대학생) 2024년 경상남도청소년참여위원회 모집',

In [16]:
# 5-2. 중복 제거
print(df.shape)
df = df.drop_duplicates()
print(df.shape)

(437, 35)
(337, 35)


In [ ]:
## 6. 중앙부처 데이터만 가져오기
# 6-1. 중앙부처/지자체 구분 확인
df['기관 및 지자체 구분'].unique()

In [17]:
# 6-2. 중앙부처 정책만 있는 df 생성
df_head = df[df['기관 및 지자체 구분']=='중앙부처']

In [20]:
# 7. csv 저장하기 
df_head.to_csv("/Users/sukyungseok/jingi-chatbot/chatbot-using-langchain/data/중앙부처.csv")